In [1]:
import py_midicsv as pm  # Documentation at https://github.com/timwedde/py_midicsv
import pandas as pd
import io
import numpy as np
from hmmlearn import hmm
# from midi2audio import FluidSynth
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from midi_tree_transform import MidiTreeTransform
import os
from keys_and_times import GetKeys, GetTimes, KeyFilter, TimeFilter

In [3]:
song_list = os.listdir('Study_Music')

transformer = MidiTreeTransform()
states, notes, lengths = transformer.read_midi_files(song_list)

KeyboardInterrupt: 

In [ ]:
time_filter = TimeFilter(transformer)
compressed_notes = time_filter.note_compress(notes)
compressed_states = transformer.create_state_array(compressed_notes)

key_class = GetKeys()
key_name = key_class.random_key(song_list)
sample_key = key_class.get_notes_from_key(key_name) 

key_filter = KeyFilter(key, transformer)
states, notes = key_filter.filter(compressed_states, compressed_notes)

In [ ]:
times = GetTimes()
song_length = times.get_sample_length(song_list) 

In [ ]:
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, verbose=True, min_samples_split=100)
rf.fit(states, notes)

curr_state = np.zeros(states.shape[1], dtype=int)

generated_notes = []
for i in tqdm(range(song_length)):
    p = rf.predict_proba(curr_state.reshape(1,-1))
    p = p[0]

    # Throttle the probability of nothing
    p[0] *= .8
    the_rest = p[1:]

    rem_prop = 1. - p[0]
    scale_fact = rem_prop/np.sum(the_rest)

    p[1:] *= scale_fact

    gen_note = np.random.choice(key_filter.filtered_dict_length, p=p)
            
    generated_notes.append(gen_note)
    
    curr_state = key_filter.update_state_array(curr_state, gen_note)

generated_notes = np.array(generated_notes)

generated_notes = key_filter.sift(generated_notes)
generated_notes = time_filter.enjoy_the_silence(generated_notes)
generated_notes

transformer.save_as_midi(generated_notes, "final_forest_song.midi")

In [3]:
notes = hmm.MultinomialHMM(n_components=20, n_iter=100, tol=0.01, verbose=True)
notes.fit(notes.reshape(-1, 1), lengths=lengths)

new_notes = notes.sample(song_length)

generated_notes = np.array(new_notes)

generated_notes = key_filter.sift(generated_notes)
generated_notes = time_filter.enjoy_the_silence(generated_notes)
generated_notes

transformer.save_as_midi(generated_notes, "final_hmm_song.midi")

         1     -694426.2221             +nan
         2      -52697.6352     +641728.5869
         3      -52694.8907          +2.7445
         4      -52691.3552          +3.5354
         5      -52686.3831          +4.9721
         6      -52678.6040          +7.7791
         7      -52664.9149         +13.6891
         8      -52639.4197         +25.4952
         9      -52598.9474         +40.4723
        10      -52552.3653         +46.5821


MultinomialHMM(n_components=20,
               random_state=RandomState(MT19937) at 0x2932D36C8C8,
               verbose=True)